[Covertype dataset](https://archive.ics.uci.edu/dataset/31/covertype) on UCI ML Repository

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

### Read the dataset

In [13]:
df = spark.read.csv("data/covertype/covtype.data.gz", header=False, inferSchema=True)
df.describe().show()

+-------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+--------------------+------------------+-------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+-------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+-------------------+-------------------+--------------------+--------------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------------+--------------------+

### Split into testing and training data

In [10]:
train_data, test_data = df.randomSplit([0.9, 0.1])
train_data.cache()
test_data.cache()

DataFrame[_c0: int, _c1: int, _c2: int, _c3: int, _c4: int, _c5: int, _c6: int, _c7: int, _c8: int, _c9: int, _c10: int, _c11: int, _c12: int, _c13: int, _c14: int, _c15: int, _c16: int, _c17: int, _c18: int, _c19: int, _c20: int, _c21: int, _c22: int, _c23: int, _c24: int, _c25: int, _c26: int, _c27: int, _c28: int, _c29: int, _c30: int, _c31: int, _c32: int, _c33: int, _c34: int, _c35: int, _c36: int, _c37: int, _c38: int, _c39: int, _c40: int, _c41: int, _c42: int, _c43: int, _c44: int, _c45: int, _c46: int, _c47: int, _c48: int, _c49: int, _c50: int, _c51: int, _c52: int, _c53: int, _c54: int]

### Collapse all those columns into a "feature vector"

In [21]:
from pyspark.ml.feature import VectorAssembler

input_cols = df.schema.names[:-1]
vector_assembler = VectorAssembler(inputCols=input_cols, outputCol="featureVector")

assembled_train_data = vector_assembler.transform(train_data)
assembled_test_data = vector_assembler.transform(test_data)

assembled_train_data.select("featureVector").show(5)

+--------------------+
|       featureVector|
+--------------------+
|(54,[0,1,2,3,4,5,...|
|(54,[0,1,2,3,4,5,...|
|(54,[0,1,2,3,4,5,...|
|(54,[0,1,2,3,4,5,...|
|(54,[0,1,2,3,4,5,...|
+--------------------+
only showing top 5 rows



### Train the model

In [18]:
from pyspark.ml.classification import DecisionTreeClassifier

labelCol = df.schema.names[-1]
classifier = DecisionTreeClassifier(seed = 1234, labelCol=labelCol, featuresCol="featureVector",
predictionCol="prediction")

model = classifier.fit(assembled_train_data)
print(model.toDebugString)

DecisionTreeClassificationModel: uid=DecisionTreeClassifier_e93505497d5f, depth=5, numNodes=43, numClasses=8, numFeatures=54
  If (feature 0 <= 3047.5)
   If (feature 0 <= 2561.5)
    If (feature 10 <= 0.5)
     If (feature 0 <= 2449.5)
      If (feature 3 <= 15.0)
       Predict: 4.0
      Else (feature 3 > 15.0)
       Predict: 3.0
     Else (feature 0 > 2449.5)
      If (feature 17 <= 0.5)
       Predict: 2.0
      Else (feature 17 > 0.5)
       Predict: 3.0
    Else (feature 10 > 0.5)
     Predict: 2.0
   Else (feature 0 > 2561.5)
    If (feature 0 <= 2952.5)
     If (feature 15 <= 0.5)
      If (feature 17 <= 0.5)
       Predict: 2.0
      Else (feature 17 > 0.5)
       Predict: 3.0
     Else (feature 15 > 0.5)
      Predict: 3.0
    Else (feature 0 > 2952.5)
     If (feature 3 <= 211.0)
      If (feature 36 <= 0.5)
       Predict: 2.0
      Else (feature 36 > 0.5)
       Predict: 1.0
     Else (feature 3 > 211.0)
      Predict: 2.0
  Else (feature 0 > 3047.5)
   If (feature 0 <= 

### Evaluate

In [30]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

predictions = model.transform(assembled_test_data)

evaluator = MulticlassClassificationEvaluator(labelCol=labelCol, predictionCol="prediction")
evaluator.evaluate(predictions)

accuracy = evaluator.setMetricName("accuracy").evaluate(predictions)
f1 = evaluator.setMetricName("f1").evaluate(predictions)

print("accuracy", accuracy)
print("f1", f1)

accuracy 0.6998868285888395
f1 0.6822383016968497


In [28]:
confusion_matrix = predictions.groupBy(labelCol) \
    .pivot("prediction") \
    .count() \
    .na.fill(0) \
    .orderBy(labelCol)

confusion_matrix.show()

+----+-----+-----+----+---+---+
|_c54|  1.0|  2.0| 3.0|4.0|7.0|
+----+-----+-----+----+---+---+
|   1|14953| 5660|  12|  0|387|
|   2| 5987|21425| 458| 16| 50|
|   3|    0|  649|2739| 74|  0|
|   4|    0|    2| 180| 89|  0|
|   5|    3|  861|  72|  0|  0|
|   6|    0|  725| 946| 75|  0|
|   7| 1073|    2|   5|  0|992|
+----+-----+-----+----+---+---+

